In [6]:
!pip install numpy==1.25

  Using cached numpy-1.25.0.tar.gz (10.4 MB)
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'error'


  error: subprocess-exited-with-error
  
  × Getting requirements to build wheel did not run successfully.
  │ exit code: 1
  ╰─> [33 lines of output]
      Traceback (most recent call last):
        File "C:\Users\iruss\AppData\Local\Programs\Python\Python312\Lib\site-packages\pip\_vendor\pyproject_hooks\_in_process\_in_process.py", line 353, in <module>
          main()
        File "C:\Users\iruss\AppData\Local\Programs\Python\Python312\Lib\site-packages\pip\_vendor\pyproject_hooks\_in_process\_in_process.py", line 335, in main
          json_out['return_val'] = hook(**hook_input['kwargs'])
                                   ^^^^^^^^^^^^^^^^^^^^^^^^^^^^
        File "C:\Users\iruss\AppData\Local\Programs\Python\Python312\Lib\site-packages\pip\_vendor\pyproject_hooks\_in_process\_in_process.py", line 112, in get_requires_for_build_wheel
          backend = _build_backend()
                    ^^^^^^^^^^^^^^^^
        File "C:\Users\iruss\AppData\Local\Programs\Python\Python312\Lib\si

In [7]:
# generate_captchas.py
import os
import random
import string
import csv
from captcha.image import ImageCaptcha
from PIL import Image
import numpy as np

# Config
OUT_DIR = "data/train_images"
CSV_PATH = "data/labels.csv"
NUM_IMAGES = 10000       # modifică după nevoie (5000, 20000, etc.)
WIDTH = 160
HEIGHT = 60
LENGTH = 8               # lungimea fiecărui captcha (ex: 8)
CHARSET = string.ascii_lowercase + string.digits  # 'abcdefghijklmnopqrstuvwxyz0123456789'
# Poți adăuga majuscule: + string.ascii_uppercase

os.makedirs(OUT_DIR, exist_ok=True)

image_gen = ImageCaptcha(width=WIDTH, height=HEIGHT)

def random_text(length=LENGTH):
    return ''.join(random.choices(CHARSET, k=length))

# Pentru reproducibilitate (opsțional)
random.seed(42)

with open(CSV_PATH, mode='w', newline='', encoding='utf-8') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(['filename', 'answer'])
    for i in range(NUM_IMAGES):
        txt = random_text()
        fname = f"train_{i:05d}.png"
        fpath = os.path.join(OUT_DIR, fname)
        # Generează imaginea
        img = image_gen.generate_image(txt)
        # Poți aplica transformări simple aici (opțional)
        # De exemplu conversie la grayscale:
        # img = img.convert('L')
        img.save(fpath)

        # Scrie în CSV
        writer.writerow([os.path.join(OUT_DIR, fname), txt])

        if (i + 1) % 500 == 0:
            print(f"Generated {i + 1}/{NUM_IMAGES}")

print("Done. Images in:", OUT_DIR)
print("CSV labels:", CSV_PATH)


Generated 500/10000
Generated 1000/10000
Generated 1500/10000
Generated 2000/10000
Generated 2500/10000
Generated 3000/10000
Generated 3500/10000
Generated 4000/10000
Generated 4500/10000
Generated 5000/10000
Generated 5500/10000
Generated 6000/10000
Generated 6500/10000
Generated 7000/10000
Generated 7500/10000
Generated 8000/10000
Generated 8500/10000
Generated 9000/10000
Generated 9500/10000
Generated 10000/10000
Done. Images in: data/train_images
CSV labels: data/labels.csv


In [8]:
import tensorflow as tf
from tensorflow.keras import layers, models, backend as K
import numpy as np
import string
import pandas as pd
from PIL import Image

# Config
IMG_WIDTH = 160
IMG_HEIGHT = 60
BATCH_SIZE = 32
EPOCHS = 30

CHARS = string.ascii_lowercase + string.digits
NUM_CLASSES = len(CHARS) + 1  # +1 pentru CTC blank

# Dicționare
char_to_num = {char: idx for idx, char in enumerate(CHARS)}
num_to_char = {idx: char for idx, char in enumerate(CHARS)}



A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.2.6 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "C:\Users\iruss\AppData\Roaming\Python\Python312\site-packages\ipykernel_launcher.py", line 18, in <module>
    app.launch_new_instance()
  File "c:\Users\iruss\AppData\Local\Programs\Python\Python312\Lib\site-packages\traitlets\config\application.py", line 1075, in launch_instance
    app.start()
  File "C:\Users\iruss\AppData\Roaming\Python\Python312\site-packages\ipykernel\kernelapp.py", line 739, in start
   

ImportError: 
A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.2.6 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.



ImportError: numpy.core._multiarray_umath failed to import

ImportError: numpy.core.umath failed to import

In [ ]:
# Imagine -> array normalizat
def preprocess_image(path):
    img = Image.open(path).convert('L')  # grayscale
    img = img.resize((IMG_WIDTH, IMG_HEIGHT))
    img = np.array(img)/255.0
    img = np.expand_dims(img, axis=-1)
    return img

# Text -> label numeric
def text_to_labels(text):
    return [char_to_num[c] for c in text]

# Label numeric -> text
def labels_to_text(labels):
    return ''.join([num_to_char[l] for l in labels if l in num_to_char])


In [ ]:
def data_generator(df, batch_size=BATCH_SIZE):
    while True:
        df = df.sample(frac=1).reset_index(drop=True)
        for i in range(0, len(df), batch_size):
            batch_df = df.iloc[i:i+batch_size]
            imgs = np.array([preprocess_image(p) for p in batch_df['filename']])
            labels = [text_to_labels(txt) for txt in batch_df['answer']]
            max_len = max(len(l) for l in labels)
            labels_padded = np.array([l + [NUM_CLASSES-1]*(max_len-len(l)) for l in labels])
            input_length = np.ones((len(imgs), 1)) * (IMG_WIDTH // 4)
            label_length = np.array([[len(l)] for l in labels])
            yield {'image_input': imgs, 'label': labels_padded,
                   'input_length': input_length, 'label_length': label_length}, np.zeros(len(imgs))


In [ ]:
def build_model(img_width=IMG_WIDTH, img_height=IMG_HEIGHT, num_classes=NUM_CLASSES):
    input_img = layers.Input(shape=(img_height, img_width, 1), name='image_input')
    labels = layers.Input(name='label', shape=(None,), dtype='float32')
    input_length = layers.Input(name='input_length', shape=(1,), dtype='int64')
    label_length = layers.Input(name='label_length', shape=(1,), dtype='int64')

    # CNN
    x = layers.Conv2D(64, (3,3), activation='relu', padding='same')(input_img)
    x = layers.MaxPooling2D((2,2))(x)
    x = layers.Conv2D(128, (3,3), activation='relu', padding='same')(x)
    x = layers.MaxPooling2D((2,2))(x)
    x = layers.Conv2D(256, (3,3), activation='relu', padding='same')(x)
    x = layers.MaxPooling2D((2,1))(x)  # păstrăm latimea pentru secvență
    x = layers.Conv2D(256, (3,3), activation='relu', padding='same')(x)
    x = layers.BatchNormalization()(x)

    # Reshape pentru RNN
    new_shape = (img_width // 4, 256)
    x = layers.Reshape(target_shape=new_shape)(x)

    # RNN
    x = layers.Bidirectional(layers.LSTM(128, return_sequences=True))(x)
    x = layers.Bidirectional(layers.LSTM(128, return_sequences=True))(x)

    # Output
    x = layers.Dense(num_classes, activation='softmax')(x)

    # Model pentru antrenament cu CTC
    def ctc_lambda_func(args):
        y_pred, labels, input_length, label_length = args
        return K.ctc_batch_cost(labels, y_pred, input_length, label_length)

    output = layers.Lambda(ctc_lambda_func, output_shape=(1,), name='ctc')([x, labels, input_length, label_length])

    model = models.Model(inputs=[input_img, labels, input_length, label_length], outputs=output)
    model.compile(optimizer='adam', loss={'ctc': lambda y_true, y_pred: y_pred})

    # Model pentru predicții
    model_pred = models.Model(inputs=input_img, outputs=x)

    return model, model_pred


In [ ]:
model, model_pred = build_model()
model.summary()


In [ ]:
df = pd.read_csv("data/labels.csv")
from sklearn.model_selection import train_test_split
train_df, val_df = train_test_split(df, test_size=0.1, random_state=42)

train_gen = data_generator(train_df, BATCH_SIZE)
val_gen = data_generator(val_df, BATCH_SIZE)


In [ ]:
steps_per_epoch = len(train_df)//BATCH_SIZE
validation_steps = len(val_df)//BATCH_SIZE

model.fit(train_gen,
          steps_per_epoch=steps_per_epoch,
          validation_data=val_gen,
          validation_steps=validation_steps,
          epochs=EPOCHS)


In [ ]:
def decode_prediction(pred):
    # pred shape: (timesteps, num_classes)
    out_best = np.argmax(pred, axis=1)
    out_best = [k for k, _ in groupby(out_best)]  # remove duplicates
    out_best = [c for c in out_best if c != NUM_CLASSES-1]  # remove CTC blank
    return labels_to_text(out_best)

# Exemplu
from itertools import groupby
test_img_path = "data/train_images/train_00000.png"
img = np.expand_dims(preprocess_image(test_img_path), axis=0)
pred = model_pred.predict(img)
text = decode_prediction(pred[0])
print("Predicted CAPTCHA:", text)
